# 패키지 설치
1. langchain
2. faiss-cpu
3. Chroma
4. python-docx
5. dotenv
6. openai

In [25]:
!pip install langchain langchain-openai langchain-chroma langchain-community openai faiss-cpu python-docx dotenv

## 환경 변수 설정

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

file_path = os.getenv("FILE_PATH")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

print(f'file_path: {file_path}')


file_path: Structural_Interpretation.docx


## 문서 로딩
- docx 문서 로딩

In [2]:
from docx import Document as DocxDocument



def load_docx(file_path) -> str:
    try:
        """
        문서 파일 읽고 텍스트로 변환
        """
        doc = DocxDocument(file_path)
        text = []
        for para in doc.paragraphs:
            text.append(para.text)
        
        print(f"loaded : {file_path}")

        return "\n".join(text)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return ""

full_text = load_docx(file_path)


loaded : Structural_Interpretation.docx


# 데이터 청킹
### 문서 구조
- [] 대괄호 category
- {} 중괄호 소분류 

해당 분류 기준을 중심으로 chunking
## WHY ? 
1. 분류를 메타데이터로 정확도 향상
2. 의미기반 chunking 으로 노이즈 없는 store구성을 위해서

In [3]:
# 문서 저장소
# 문서는 [] 로 대주제, {}로 소주제로 나뉘어져 있음 기준으로 청킹
import re
from langchain_core.documents import Document
from pprint import pprint

chunks = [] # 청크 저장소
main_topic = None # 대주제
sub_topic = None # 소주제
content = [] # 본문

def add_chunk(main, sub, content):
    """
    청크를 추가하는 함수
    """
    if main and content:
        chunks.append(
            Document(
                page_content=" ".join(content).strip(),
                metadata={"main_topic": main, "sub_topic": sub},
            )
        )

# 청킹
"""
각 주제가 바뀌는 [], {} 단위로 청크를 나누고 저장
문서 구조) 
[대주제1]
{소주제1}
내용1
내용2
{소주제2}
내용3

"""
try:
    for line in full_text.split("\n"):
        line = line.strip()
        # 빈줄
        if not line:
            continue
        # 대주제
        if re.match(r"^\[.*\]$", line):
            # 이전 청크 저장
            add_chunk(main_topic, sub_topic, content)

            content = []
            main_topic = line[1:-1].strip()
            sub_topic = None
            continue
        # 소주제
        if re.match(r"^\{.*\}$", line):
            # 이전 청크 저장
            add_chunk(main_topic, sub_topic, content)
            content = []
            sub_topic = line[1:-1].strip()
            continue
        # 내용
        content.append(line)
    # 마지막 청크 저장
    add_chunk(main_topic, sub_topic, content)
    print(f"Total chunks: {len(chunks)}")
    print("✅ done")
except Exception as e:
    print(f"Error during chunking: {e}")


Total chunks: 77
✅ done


# 청크 임베딩
## vectorDB
### FAISS, ChromaDB

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma

# 임베딩모델 설정
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)


In [14]:
# %%time
# 벡터 저장소 로드
import time

batch_size = 10  # 한번에 처리할 chunk 개수 (필요시 조절)
all_chunks = chunks  # chunks는 이미 정의되어 있는 리스트

# faiss_store
faiss_store = FAISS.load_local("faiss_store", embeddings=embeddings, allow_dangerous_deserialization=True) if os.path.exists("faiss_store") else None

if faiss_store is None:
    print("FAISS store 생성 시작")
    for i in range(0, len(all_chunks), batch_size):
        batch = all_chunks[i:i + batch_size]
        # 벡터스토어 생성 및 추가
        if faiss_store is None:
            faiss_store = FAISS.from_documents(batch, embedding=embeddings)
        else:
            faiss_store.add_documents(batch)
        # 요청 속도 제한에 대비하여 딜레이 추가
        time.sleep(1)
    else:
        print("생성 완료")
else:
    print("FAISS store 로드 완료")

# chroma_store
chroma_store = Chroma(persist_directory="chroma_store", embedding_function=embeddings) if os.path.exists("chroma_store") else None

if chroma_store is None:
    print("ChromaDB 생성 시작")
    for i in range(0, len(all_chunks), batch_size):
        batch = all_chunks[i:i + batch_size]
        # 벡터스토어 생성 및 추가
        if chroma_store is None:
            chroma_store = Chroma.from_documents(batch, embedding=embeddings, persist_directory="chroma_store")
        else:
            chroma_store.add_documents(batch)
        # 요청 속도 제한에 대비하여 딜레이 추가
        time.sleep(1)
    else:
        print("생성 완료")

else:
    print("ChromaDB 로드 완료")
        
    
print("✅done")

FAISS store 로드 완료
ChromaDB 로드 완료
✅done


### TEST

In [18]:
# 모든 청크의 내용과 메타데이터 확인
# for doc in chroma_store.similarity_search("오른쪽으로 치우쳐짐", k=4):
#     print(doc.page_content, doc.metadata)

print(chroma_store._collection.count())



77


# save store

In [19]:
faiss_store.save_local("faiss_store")